In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

# from train_evaluate import NMR_prediction

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.create_node_embedding.node_embeddings_godess import create_node_embeddings

from preprocess.build_adjaency_matrix.create_adjaency_matrix_godess import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_godess import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs


##### 1, Create Node embeddings




##### Initialize atom-level node embeddings based on extracted atom and monosaccharide features. 

1, atom name: atom name that indicate ring position, Eg: 04, C4, C3...

2, bound_orig: monosaccharide's bound information from original pdb file.

3, atom_type: general atom type Eg. O, C, H, N.

4, ab_list: anomeric orientation of hydroxyl group.

5, dl_list: fischer convention, fischer d, l.

6, pf_list: number of carbon p, f.

7, mono_accurate_list: accurate monosaccharide name, a-D-Glcp.

8, mono_simple_list: simple monosaccharide name, glc.

9, me_list: Me root interatction.

10, ser_list: Ser root interaction.

11, s_list: Sulfur residue effect.

12, ac_list: Ac residue effect.

13, gc_list: Gc residue effect.

In [3]:
# data directory
data_dir = 'godess/data/'

# define the lengtht of the embeddings of each feature
atom_name_dim=256

bound_orig_dim=32

atom_type_dim=32

ab_dim=64

dl_dim=64

pf_dim=64

mono_accurate_dim=128

mono_simple_dim=256

me_dim=32

ser_dim=32

s_dim=64

ac_dim=64

gc_dim=32

In [4]:


C = create_node_embeddings(data_dir = data_dir)
C.write_all_embeddings(atom_name_dim = atom_name_dim, bound_orig_dim = bound_orig_dim, 
                           atom_type_dim = atom_type_dim, ab_dim = ab_dim, dl_dim = dl_dim,
                           pf_dim = pf_dim, mono_accurate_dim = mono_accurate_dim, 
                           mono_simple_dim = mono_simple_dim, me_dim=me_dim, 
                          ser_dim=ser_dim, s_dim=s_dim, ac_dim=ac_dim, gc_dim=gc_dim)

100%|██████████████████████████████████████| 2310/2310 [00:05<00:00, 403.34it/s]


##### 2, Build Adjaency matrix

##### Build adjaency matrix based on threshold distance between atoms

1, threshold_carbon: we construct edges between two carbon atom if their distance is within 1.65Å. 

2, threshold_hydrogen: we construct edges between two hydrogen atom if their distance is within 1.18Å. 

3, threshold_general: for all other atoms, we construct edges between distance two atom is within 1.5Å. 

4, threshold_interaction: we construct interaction edges between atoms if their distance is with in 5Å. 


In [5]:
threshold_carbon=1.65

threshold_hydrogen=1.18

threshold_general=1.5

threshold_interaction=5.0

In [6]:
B = build_adjacency_matrix(labeled_pdb_dir = data_dir,
                           threshold_carbon=threshold_carbon,
                           threshold_hydrogen=threshold_hydrogen, 
                           threshold_general=threshold_general, 
                           threshold_interaction=threshold_interaction)
B.calculate_all_matrix()

100%|███████████████████████████████████████| 2310/2310 [01:35<00:00, 24.27it/s]


###### 3, Create glycan graph

##### create glycan graph from prebuilt node embeddings and adjaency matrix, every glycan is considered as a graph and every atom is consided as a node

1, num_test: number of testing glycans, we randomly select 20% of the glycans as test set

In [7]:
num_test = 462
Create = create_graph(data_dir=data_dir, num_test=num_test)
g, test_index = Create.create_all_graph()

--------------------------loading NMR Graph-------------------------------


100%|███████████████████████████████████████| 2310/2310 [04:11<00:00,  9.17it/s]


###### 4, Train a simple gcn model


In [7]:
g.ndata['z']
g.ndata['train_carbon_mask']

tensor([False, False, False,  ..., False, False, False])

In [8]:
in_size = atom_name_dim + ab_dim + dl_dim + \
            pf_dim + mono_simple_dim + me_dim + ser_dim + \
            s_dim + ac_dim + gc_dim

hidden_size_1 = int(in_size / 2)
hidden_size_2 = int(hidden_size_1 / 2)
hidden_size_3 = 256
hidden_size_4 = 128

num_epoch = 2000
lr = 1e-3

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)

features = g.ndata['z']

labels = g.ndata['y']

masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']

model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)

NMR_prediction = NMR_prediction(results_dir='godess/results/training_carbon.csv',
                                results_dir_test = 'godess/results/testing_carbon.csv',
                                model_dir='godess/results/Model_Godess_carbon.pt',
                                num_epoch = num_epoch, 
                                lr = lr)

print("Training...")

NMR_prediction.train(g, features, labels, masks, model)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))

Training...


  0%|                                          | 2/2000 [00:00<14:20,  2.32it/s]

18610
75422
Epoch 00000 | Loss 6716.6646 | train_RMSE 79.0701 | test_RMSE 79.4029 
18610
75422
Epoch 00001 | Loss 6252.1514 | train_RMSE 73.3460 | test_RMSE 73.7184 


  0%|                                          | 4/2000 [00:01<08:13,  4.05it/s]

18610
75422
Epoch 00002 | Loss 5380.3237 | train_RMSE 62.7878 | test_RMSE 63.2359 
18610
75422
Epoch 00003 | Loss 3945.3054 | train_RMSE 45.5976 | test_RMSE 46.1762 


...

18610
75422
Epoch 01995 | Loss 15.9167 | train_RMSE 1.9025 | test_RMSE 1.8991 
18610
75422
Epoch 01996 | Loss 15.7201 | train_RMSE 1.8361 | test_RMSE 1.8388 


100%|██████████████████████████████████████▉| 1999/2000 [05:08<00:00,  6.44it/s]

18610
75422
Epoch 01997 | Loss 15.4582 | train_RMSE 1.8314 | test_RMSE 1.8330 
18610
75422
Epoch 01998 | Loss 15.8676 | train_RMSE 1.8850 | test_RMSE 1.8825 


100%|███████████████████████████████████████| 2000/2000 [05:08<00:00,  6.48it/s]


18610
75422
Epoch 01999 | Loss 15.6345 | train_RMSE 1.8960 | test_RMSE 1.9008 
best loss: 15.458209991455078
Testing...
75422
18610
train RMSE 1.8314
test RMSE 1.8330
